# Water Surface Extraction – Laguna de Fúquene (1985–2024)

This notebook extracts and quantifies the annual surface area of the Laguna de Fúquene using satellite imagery. The output of this notebook is a tabular dataset that will be used in subsequent analysis notebooks.

The focus here is **data extraction and methodological transparency**, not interpretation or trend analysis.

## 1. Objective

The objective of this notebook is to:
- Extract annual water surface area estimates for the Laguna de Fúquene
- Ensure temporal consistency across the full historical period (1985–2024)
- Generate a reproducible dataset suitable for downstream analysis

No conclusions or trend interpretations are produced at this stage.

## 2. Libraries and Earth Engine initialization

In [ ]:
import ee
import pandas as pd
import numpy as np

ee.Initialize()

## 3. Area of study

The region of interest (ROI) corresponds to the Laguna de Fúquene, Colombia. A polygon covering the full historical extent of the lake is used to avoid systematic underestimation of surface area.

In [ ]:
# ROI definition (example placeholder)
roi = ee.Geometry.Polygon([
    [[-73.78, 5.52], [-73.78, 5.40], [-73.60, 5.40], [-73.60, 5.52]]
])

## 4. Satellite data selection

This analysis relies on Landsat surface reflectance products to ensure long-term temporal consistency. Landsat 5, 7, and 8 collections are used depending on the year.

In [ ]:
def get_landsat_collection(start_date, end_date):
    collection = (
        ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2'))
        .merge(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'))
        .filterDate(start_date, end_date)
        .filterBounds(roi)
    )
    return collection

## 5. Water detection method

Water surfaces are identified using a spectral water index (e.g., NDWI). A fixed threshold is applied consistently across all years to avoid introducing subjective adjustments.

In [ ]:
def add_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    return image.addBands(ndwi)

## 6. Annual water surface area calculation

For each year, images are aggregated and the total surface area classified as water is calculated in hectares.

In [ ]:
years = list(range(1985, 2025))
water_area_data = []

for year in years:
    start = f'{year}-01-01'
    end = f'{year}-12-31'

    collection = get_landsat_collection(start, end).map(add_ndwi)
    composite = collection.median()

    water_mask = composite.select('NDWI').gt(0)

    area_image = water_mask.multiply(ee.Image.pixelArea())
    area = area_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=30,
        maxPixels=1e13
    )

    area_ha = ee.Number(area.get('NDWI')).divide(10000)

    water_area_data.append({
        'year': year,
        'area_ha': area_ha.getInfo()
    })

## 7. Output dataset

The final output of this notebook is a CSV file containing annual water surface area estimates. This dataset serves as the input for all subsequent analysis notebooks.

In [ ]:
df = pd.DataFrame(water_area_data)

df.to_csv(
    'data/processed/water_surface_area_fquene_1985_2024.csv',
    index=False
)

df.head()